In [1]:
import kaggle

In [6]:
!kaggle datasets download -d adityajn105/flickr8k --debug

usage: kaggle [-h] [-v] [-W]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...
kaggle: error: unrecognized arguments: --debug
